In [20]:
from collections import defaultdict, Counter
import pandas as pd
from bert_score import score as bert_score
from nlgeval import NLGEval
import seaborn as sns
from nltk import ngrams

In [21]:
df = pd.concat([
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_23june/adaptive_speak_eval_out_domain_appliances.csv'),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_23june/adaptive_speak_eval_out_domain_food.csv'),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_23june/adaptive_speak_eval_out_domain_indoor.csv'),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_23june/adaptive_speak_eval_out_domain_outdoor.csv'),
    # pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_22june/adaptive_speak_eval_vehicles_v1.csv')
])

# pd.to_numeric(df['target domain'], errors='coerce')
df = df.dropna()
print('target domain   :', df['target domain'].unique())
print('listener domain :', df['listener domain'].unique())
print('simulator domain:', df['simulator domain'].unique())
print('Data points:', len(df))

df.head(2)


target domain   : ['food' 'vehicles' 'outdoor' 'indoor' 'appliances']
listener domain : ['appliances' 'food' 'indoor' 'outdoor']
simulator domain: ['appliances' 'food' 'indoor' 'outdoor']
Data points: 27907


,Unnamed: 0,target domain,listener domain,simulator domain,target img idx,img path #0,img path #1,img path #2,img path #3,img path #4,...,adapted_list_out_s0,adapted_list_out_s1,adapted_list_out_s2,original_sim_out,adapted_sim_out_s0,adapted_sim_out_s1,original_acc,adapted_accuracy_s0,adapted_accuracy_s1,adapted_accuracy_s2
0,0,food,appliances,appliances,5,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,"[[13.496817588806152], [14.18220043182373], [1...","[[13.496817588806152], [14.18220043182373], [1...","[[13.496817588806152], [14.18220043182373], [1...","[[22.605070114135742], [23.6052188873291], [22...","[[22.604656219482422], [23.604846954345703], [...","[[22.604257583618164], [23.604488372802734], [...",0.0,0.0,0.0,0.0
1,1,food,appliances,appliances,0,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,"[[10.616264343261719], [11.301734924316406], [...","[[10.616264343261719], [11.301734924316406], [...","[[10.616264343261719], [11.301734924316406], [...","[[22.24153709411621], [22.164512634277344], [2...","[[22.241954803466797], [22.164031982421875], [...","[[22.242359161376953], [22.163572311401367], [...",0.0,0.0,0.0,0.0


In [22]:
refs = df['golden utt'].tolist()
refs_for_nlgeval = [[ref] for ref in refs]
hyps = {
    "h0": df['original utt'].tolist(),
    "h0' 0": df['adapted utt s0'].tolist(),
    "h0' 1": df['adapted utt s1'].tolist(),
    "h0' 2": df['adapted utt s2'].tolist()
}


## Classic NLG metrics

In [ ]:
nlgeval = NLGEval(no_skipthoughts=True, no_glove=True, metrics_to_omit=['METEOR'])

score_dicts = []
for hyp_type in hyps:
    metrics_dict = nlgeval.compute_metrics(refs_for_nlgeval, hyps[hyp_type])
    (P, R, F)= bert_score(
        hyps[hyp_type],
        refs,
        lang="en",
        return_hash=False,
        model_type="microsoft/deberta-xlarge-mnli",
    )
    metrics_dict['BERT_P'] = P.mean().item()
    metrics_dict['BERT_R'] = R.mean().item()
    metrics_dict['BERT_F'] = F.mean().item()
    score_dicts.append(metrics_dict)

score_df = pd.DataFrame(score_dicts, index=hyps.keys())

Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.bias', 'pooler.dense.weight', 'classifier.weight', 'pooler.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
axes = score_df.plot.bar(rot=0, subplots=True, figsize=(10,20), legend=None)
# axes[1].legend()

## Utterance length

In [ ]:
lengths = defaultdict(list)

for ref in refs:
    lengths['ref'].append(len(ref.strip().split()))

for hyp_type in hyps:
    for hyp in hyps[hyp_type]:
        lengths[hyp_type].append(len(hyp.strip().split()))

len_df = pd.DataFrame(lengths.values(), index=lengths.keys()).T


In [ ]:
sns.violinplot(data=len_df)

## Repetitions

In [ ]:
def ngram_reps(text, n=1):
    token_seq = text.strip().split()
    ngram_seq = list(ngrams(token_seq, n))
    if len(ngram_seq) <= 1:
        return 0.

    ngram_counter = Counter(ngram_seq)
    n_repeated_tokens = 0
    n_repeated_types = 0
    for w in ngram_counter:
        if ngram_counter[w] > 1:
            n_repeated_tokens += ngram_counter[w]
            n_repeated_types += 1

    return (n_repeated_tokens - n_repeated_types) / len(ngram_seq)


In [ ]:
reps = defaultdict(lambda: defaultdict(list))

for N in range(1, 4):
    for ref in refs:
        reps[N]['ref'].append(ngram_reps(ref, n=N))

    for hyp_type in hyps:
        for hyp in hyps[hyp_type]:
            reps[N][hyp_type].append(ngram_reps(hyp, n=N))

rep_df_1 = pd.DataFrame(reps[1].values(), index=reps[1].keys()).T
rep_df_2 = pd.DataFrame(reps[2].values(), index=reps[2].keys()).T
rep_df_3 = pd.DataFrame(reps[3].values(), index=reps[3].keys()).T

In [ ]:
sns.violinplot(data=rep_df_1)

In [ ]:
sns.violinplot(data=rep_df_2, inner=None, color=".8")
sns.stripplot(data=rep_df_2)

In [ ]:
sns.violinplot(data=rep_df_3, inner=None, color=".8")
sns.stripplot(data=rep_df_3)